## Dataset

[IDS 2018 dataset](https://www.unb.ca/cic/datasets/ids-2018.html). Dataset contains multiple different attacks and has network PCAP files and operating system logs from Ubuntu and Windows. Focus now on Windows EVTX logs.

## Threat Hunting
Following the document structure as specified by Robert Rodriguez in [Threat Hunter's Playbook](https://threathunterplaybook.com/introduction.html)

### Hypothesis

Suspicious possible infiltration phase activity detected by automated IDS during 28.02 - 02.03

### Technical Context

In this scenario, a vulnerable application (such as Adobe Acrobat Reader 9) should be exploited. First the victim receives a malicious document through the email. Then, after successful exploitation using Metasploit framework, a backdoor will be executed on the victim’s computer. Now we can conduct different attacks on the victim’s network include IP sweep, full port scan and service enumerations using Nmap.

### Tools
* Dataset's Windows EVTX logs uploaded to Elasticsearch
    * Using Python library [evtxtoelk](https://pypi.org/project/evtxtoelk/)
* Query the Elasticsearch data 
    * using Elastic's Python library [Eland](https://github.com/elastic/eland)
    * Uses DataFrame structure similar to pandas
    * Easily filter and query large amounts of data without needing to load to local memory
* After data is filtered in Eland, load locally to CSV - file and to Pandas DataFrame
* Filter local dataframe to find infiltration events

## Analytics

Import necessary libraries and define Elasticsearch location, index

In [1]:
import datetime
import eland as ed
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None # Show all columns when DataFrame is printed in output.

index = 'ids'
es_host = '192.168.85.12:9200'
df = ed.DataFrame(es_host,index)
df.head()

,@timestamp,@xmlns,Event.@xmlns,Event.EventData.@Name,Event.EventData.Binary,Event.EventData.Data.AccountName,Event.EventData.Data.AdapterName,Event.EventData.Data.Address,Event.EventData.Data.AddressLength,Event.EventData.Data.AlertDesc,...,UserData.UMDFReflectorDependencyMissing.@xmlns:auto-ns2,UserData.UMDFReflectorDependencyMissing.Dependency,UserData.UMDFServiceInstall.@upgrade,UserData.UMDFServiceInstall.@xmlns,UserData.UMDFServiceInstall.@xmlns:auto-ns2,UserData.UMDFServiceInstall.MinimumFxVersion,UserData.UMDFServiceInstall.Service.@clsid,UserData.UMDFServiceInstall.Service.@name,UserData.updatelist.#text,UserData.updatelist.@xmlns
xKflEHgBICROT9KQt2Gl,2018-02-13 02:30:46.327957,NaN,http://schemas.microsoft.com/win/2004/08/event...,NaN,QQBtAGEAegBvAG4AUwBTAE0AQQBnAGUAbgB0AC8AMQAAAA==,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KKflEHgBICROT9KQt2CW,2018-01-23 23:50:30.582506,NaN,http://schemas.microsoft.com/win/2004/08/event...,NaN,NaN,NaN,NaN,fe80::5efe:172.31.64.73,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xqflEHgBICROT9KQt2Gl,2018-02-13 02:30:46.327957,NaN,http://schemas.microsoft.com/win/2004/08/event...,NaN,QQBtAGEAegBvAG4AUwBTAE0AQQBnAGUAbgB0AAAA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x6flEHgBICROT9KQt2Gl,2018-02-13 02:33:09.719076,NaN,http://schemas.microsoft.com/win/2004/08/event...,NaN,QQBtAGEAegBvAG4AUwBTAE0AQQBnAGUAbgB0AC8AMQAAAA==,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yKflEHgBICROT9KQt2Gl,2018-02-13 02:33:09.719076,NaN,http://schemas.microsoft.com/win/2004/08/event...,NaN,QQBtAGEAegBvAG4AUwBTAE0AQQBnAGUAbgB0AAAA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filter Eland DataFrame by timestamp before downloading it to local CSV

In [2]:
## Filter eland Dataframe (using es query)
query = {
    "range": {
        "@timestamp": {
            "gte": "2018-02-28",
            "lte": "2018-03-02"
        }
    }
}
df2 = df.es_query(query)
df2.to_csv('02-28_03-02.csv',index=False)

Load to local Pandas DataFrame from downloaded CSV, clean DataFrame by dropping NaN and None values

In [3]:
df_pd = pd.read_csv('02-28_03-02.csv')
df_pd['@timestamp'] = pd.to_datetime(df_pd['@timestamp'])
df_pd = df_pd.set_index('@timestamp')
df_pd = df_pd.replace("None",np.nan) # convert string "None" to NaN
df_pd = df_pd.dropna(how='all',axis=1)
df_pd.index = df_pd.index.round('S')
df_pd = df_pd.sort_index()

C:\Users\Joni\anaconda3\envs\vega\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,5,6,7,14,15,16,17,19,22,25,26,29,33,34,35,39,42,43,45,47,49,50,52,72,75,78,82,83,84,88,92,94,114,115,117,120,126,131,132,133,139,140,142,147,150,153,156,158,159,161,162,164,165,166,167,169,170,172,175,178) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Event IDs to filter

The next step is to gather a list of Event IDs that are used to filter the log data.

### Mitre ATT&CK matrix
Event IDs gathered from PHR model's Data Collection section.
* Command and Control
    * Event ID 3: Network connection Sysmon can be used to monitor process network connection events (ID 3). These events should be filtered by process name or destination IP to only include suspicious network activity, such as processes that shouldn't normally communicate to Internet or that are communicating with unusual destinations.
* Credential Access
    * Event ID 12: RegistryEvent (Object create and delete) Windows stores credential provider definitions in registry location: HKLM\Software\Microsoft\Windows\CurrentVersion\Authentication\CredentialProviders Creation of new credential provider can be detected by monitoring Sysmon registry modification events for the CredentialProviders location.
* Defense Evasion
    * Event ID 23: FileDelete. Sysmon generates event ID 23 when a file is deleted.
    * Event ID 4103: Module Logging. PowerShell can interpret commands encoded using the base64-encoding. PowerShell module logging (ID 4103) records the options used with execution as well as de-obfuscated commands.
* Discovery
    * Event ID 1: Process creation. Tools used in this technique can be detected by monitoring the specific process command-line arguments (e.g. ipconfig or route print) from Sysmon Event ID 1.
* Execution
    * Event ID 4104: Script Block Logging. Script block logging records blocks of code as they are executed by the PowerShell engine, capturing the full context of the executed code, including scripts and commands.
* Lateral Movement
    * Event ID 4624: An account was successfully logged on.
    * Event ID 2624: Successful authentication using remote desktop connection. The logon type 10 (RemoteInteractive) indicates that the user logged in using remote desktop connection.
    * Event ID 5140: A network share object was accessed. The use of this technique can be detected by monitoring the event ID 5140 and filtering specifically for share names that match the common admin share names.
* Persistence
    * Event ID 13: RegistryEvent (Value Set). Detecting the use of registry run keys requires monitoring changes to the relevant registry keys.
    * Event ID 4698: A scheduled task was created. Event ID 4702: A scheduled task was updated. Windows generates event ID 4698 when at.exe is used to schedule task. 
    * Event ID 4702: A scheduled task is updated.
* Privilege Escalation: 
    * Event ID 7045: A new service was installed in the system. The event ID 7045 is generated in all modern Windows versions when a new service is created.
    * Event ID 4697: A service was installed in the system, which is generated in newer versions of Windows (Windows 10 and Server 2016).
    
### Powershell attacks

PowerShell attacks, Event ID:s to look out for:
* Event ID 7040 “The start type of the Windows Remote Management (WS-Management) service was changed from [disabled / demand start] to auto start.” – recorded when PowerShell remoting is enabled.
* Event ID 10148 (“The WinRM service is listening for WS-Management requests”) – recorded upon reboot on systems where remoting has been enabled.
* Event ID 169 (“User [DOMAIN\Account] authenticated successfully using [authentication_protocol]”)
* Event ID 4688 (“A new process has been created”) – includes account name, domain, and executable name in the event message.

In [4]:
event_id_to_monitor = [1,3,12,23,169,2624,4103,4104,4624,4688,4697,4698,4702,5140,7040,7045,10148] # filter by all IDs listed earlier
filtered_df = df_pd[df_pd["Event.System.EventID.#text"].isin(event_id_to_monitor)].dropna(how='all',axis=1)
# filter relevenat columns
columns_to_monitor = ['Event.EventData.Data.ImagePath','Event.EventData.Data.ProcessPath','Event.EventData.Data.StartType',
                      'Event.EventData.Data.param1','Event.EventData.Data.param2','Event.EventData.Data.param3','Event.EventData.Data.param4',
                      'Event.System.Computer','Event.System.EventID.#text','Event.System.Provider.@Name','Event.System.Security.@UserID','Event.System.TimeCreated.@SystemTime']
filtered_df[columns_to_monitor]

,Event.EventData.Data.ImagePath,Event.EventData.Data.ProcessPath,Event.EventData.Data.StartType,Event.EventData.Data.param1,Event.EventData.Data.param2,Event.EventData.Data.param3,Event.EventData.Data.param4,Event.System.Computer,Event.System.EventID.#text,Event.System.Provider.@Name,Event.System.Security.@UserID,Event.System.TimeCreated.@SystemTime
@timestamp,,,,,,,,,,,,
2018-02-28 03:08:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WIN-J6GMIG1DQE5,3,Virtual Disk Service,NaN,2018-02-28 03:08:13.000000
2018-02-28 03:08:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WIN-J6GMIG1DQE5,3,Virtual Disk Service,NaN,2018-02-28 03:08:13.000000
2018-02-28 03:08:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WIN-J6GMIG1DQE5,3,Virtual Disk Service,NaN,2018-02-28 03:08:13.000000
2018-02-28 06:04:39,C:\ProgramData\Microsoft\Windows Defender\Defi...,NaN,system start,NaN,NaN,NaN,NaN,PC1,7045,Service Control Manager,S-1-5-18,2018-02-28 06:04:39.371071
2018-02-28 06:04:39,C:\ProgramData\Microsoft\Windows Defender\Defi...,NaN,system start,NaN,NaN,NaN,NaN,PC1,7045,Service Control Manager,S-1-5-18,2018-02-28 06:04:39.371071
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-02 19:35:20,C:\ProgramData\Microsoft\Windows Defender\Defi...,NaN,system start,NaN,NaN,NaN,NaN,EC2AMAZ-O4EL3NG,7045,Service Control Manager,S-1-5-18,2018-03-02 19:35:19.870102
2018-03-02 19:57:49,C:\ProgramData\Microsoft\Windows Defender\Defi...,NaN,system start,NaN,NaN,NaN,NaN,EC2AMAZ-O4EL3NG,7045,Service Control Manager,S-1-5-18,2018-03-02 19:57:49.062212
2018-03-02 21:07:29,C:\ProgramData\Microsoft\Windows Defender\Defi...,NaN,system start,NaN,NaN,NaN,NaN,EC2AMAZ-O4EL3NG,7045,Service Control Manager,S-1-5-18,2018-03-02 21:07:28.896566


A lot of Windows Defender update paths in ImagePath columns, we can use FuzzyWuzzy to compute strings using Levenhstein distance to find the strings most different from Defender image path.

In [37]:
from fuzzywuzzy import process
not_defender = process.extract("Windows Defender",filtered_df["Event.EventData.Data.ImagePath"].dropna().values,limit=len(filtered_df))
not_defender = sorted(not_defender, key = lambda tupl: tupl[1])
not_defender[:20]

[('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /svc', 27),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /svc', 27),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /svc', 27),
 ('"C:\\Program Files\\AVAST Software\\Avast\\AvastSvc.exe"', 27),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /svc', 27),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /svc', 27),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /svc', 27),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /medsvc', 32),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /medsvc', 32),
 ('"C:\\Program Files (x86)\\Google\\Update\\GoogleUpdate.exe" /medsvc', 32),
 ('\\SystemRoot\\System32\\drivers\\loop.sys', 32),
 ('\\SystemRoot\\System32\\drivers\\loop.sys', 32),
 ('\\SystemRoot\\System32\\drivers\\loop.sys', 32),
 ('\\SystemRoot\\system32\\DRIVERS\\npcap.sys', 32),
 ('\\SystemRoot\\system32\\DRIVERS\\npca

NPCap driver accessed, we can get additional information from the logs:

In [28]:
filtered_df[filtered_df["Event.EventData.Data.ImagePath"] == '\\SystemRoot\\system32\\DRIVERS\\npcap.sys'].dropna(how='all',axis=1)

,Event.@xmlns,Event.EventData.Data.ImagePath,Event.EventData.Data.ServiceName,Event.EventData.Data.ServiceType,Event.EventData.Data.StartType,Event.System.Channel,Event.System.Computer,Event.System.EventID.#text,Event.System.EventID.@Qualifiers,Event.System.EventRecordID,Event.System.Execution.@ProcessID,Event.System.Execution.@ThreadID,Event.System.Keywords,Event.System.Level,Event.System.Opcode,Event.System.Provider.@EventSourceName,Event.System.Provider.@Guid,Event.System.Provider.@Name,Event.System.Security.@UserID,Event.System.Task,Event.System.TimeCreated.@SystemTime,Event.System.Version
@timestamp,,,,,,,,,,,,,,,,,,,,,,
2018-02-28 13:42:16,http://schemas.microsoft.com/win/2004/08/event...,\SystemRoot\system32\DRIVERS\npcap.sys,Npcap Packet Driver (NPCAP),kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,25343,720.0,2872.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 13:42:15.597319,0.0
2018-02-28 13:42:16,http://schemas.microsoft.com/win/2004/08/event...,\SystemRoot\system32\DRIVERS\npcap.sys,Npcap Packet Driver (NPCAP),kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,25343,720.0,2872.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 13:42:15.597319,0.0
2018-02-28 13:42:16,http://schemas.microsoft.com/win/2004/08/event...,\SystemRoot\system32\DRIVERS\npcap.sys,Npcap Packet Driver (NPCAP),kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,25343,720.0,2872.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 13:42:15.597319,0.0
2018-02-28 14:34:39,http://schemas.microsoft.com/win/2004/08/event...,\SystemRoot\system32\DRIVERS\npcap.sys,Npcap Packet Driver (NPCAP),kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,61088,720.0,4944.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 14:34:39.310961,0.0
2018-02-28 14:34:39,http://schemas.microsoft.com/win/2004/08/event...,\SystemRoot\system32\DRIVERS\npcap.sys,Npcap Packet Driver (NPCAP),kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,61088,720.0,4944.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 14:34:39.310961,0.0
2018-02-28 14:34:39,http://schemas.microsoft.com/win/2004/08/event...,\SystemRoot\system32\DRIVERS\npcap.sys,Npcap Packet Driver (NPCAP),kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,61088,720.0,4944.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 14:34:39.310961,0.0


The UserID behind the event has 18 events attributed to it, while majority events are from S-1-5-18 and S-1-5-19

In [30]:
filtered_df['Event.System.Security.@UserID'].value_counts()

S-1-5-18                                          10361
S-1-5-19                                           1763
S-1-5-21-2491872488-200920736-2214820935-500         20
S-1-5-21-2658501782-1119487302-2855053826-500        18
S-1-5-21-2982620001-2554657312-1757520227-1001        4
S-1-5-21-2658501782-1119487302-2855053826-1000        3
S-1-5-21-2491872488-200920736-2214820935-1002         2
Name: Event.System.Security.@UserID, dtype: int64

In [33]:
filtered_df[filtered_df['Event.System.Security.@UserID'] == 'S-1-5-21-2658501782-1119487302-2855053826-500'].dropna(how='all',axis=1)

,Event.@xmlns,Event.EventData.Data.AccountName,Event.EventData.Data.ImagePath,Event.EventData.Data.NewTime,Event.EventData.Data.OldTime,Event.EventData.Data.Reason,Event.EventData.Data.ServiceName,Event.EventData.Data.ServiceType,Event.EventData.Data.StartType,Event.System.Channel,Event.System.Computer,Event.System.EventID.#text,Event.System.EventID.@Qualifiers,Event.System.EventRecordID,Event.System.Execution.@ProcessID,Event.System.Execution.@ThreadID,Event.System.Keywords,Event.System.Level,Event.System.Opcode,Event.System.Provider.@EventSourceName,Event.System.Provider.@Guid,Event.System.Provider.@Name,Event.System.Security.@UserID,Event.System.Task,Event.System.TimeCreated.@SystemTime,Event.System.Version
@timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswHwid.sys,NaN,NaN,NaN,aswHwid,kernel mode driver,demand start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54488,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.881098,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswStm.sys,NaN,NaN,NaN,aswStm,kernel mode driver,auto start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54493,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.896723,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswVmm.sys,NaN,NaN,NaN,aswVmm,kernel mode driver,boot start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54492,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.896723,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswMonFlt.sys,NaN,NaN,NaN,aswMonFlt,kernel mode driver,auto start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54489,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.881098,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswRdr2.sys,NaN,NaN,NaN,aswRdr,kernel mode driver,system start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54486,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.877136,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswSP.sys,NaN,NaN,NaN,aswSP,kernel mode driver,system start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54491,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.881098,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswArPot.sys,NaN,NaN,NaN,aswArPot,kernel mode driver,system start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54487,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826-500,0,2018-02-28 12:52:52.877136,0.0
2018-02-28 12:52:53,http://schemas.microsoft.com/win/2004/08/event...,NaN,C:\Windows\system32\drivers\aswHdsKe.sys,NaN,NaN,NaN,aswHdsKe,kernel mode driver,system start,System,EC2AMAZ-O4EL3NG,7045,16384.0,54484,724.0,776.0,0x8080000000000000,4,0.0,Service Control Manager,{555908d1-a6d7-4695-8e1e-26931d2012f4},Service Control Manager,S-1-5-21-2658501782-1119487302-2855053826

### Hunter Notes

* NPCap packet driver **Event.EventData.Data.ImagePath \SystemRoot\system32\DRIVERS\npcap.sys** started at **2018-02-28 13:42**, **Event ID 7045**,*started by **UserID S-1-5-21-2658501782-1119487302-2855053826-500**
* Could be a beginning stage of infiltration where the attacker is collecting data about the network that the victim machine **EC2AMAZ-O4EL3NG** is connected in.
* Further investigation needed into victim machine and the one's it has communicated with.

## References
* Threat Hunter's Playbook https://threathunterplaybook.com/introduction.html
* Investigating PowerShell Attacks https://www.powershellmagazine.com/2014/07/16/investigating-powershell-attacks/